# Summarize singscore accuracy by sample

While our signature was able to successfully predict many samples well, even those it had never seen before, many samples were miscategorized.

Here, we will summarize predictions for all wildtype and bortezomib resistant profiles.

There are four categories, with some overlap.

1. High confidence predictions (greater than or less than the permuted Singscore, depending on status)
2. Accurate predictions (greater than or equal to zero, depending on status)
3. Inaccurate predictions (greater than or equal to zero in the opposite direction, depending on status)
4. Completely wrong predictions (greater or less than the permuted Singscore in the opposite direction, depending on status)

For example, a resistant profile with score 1.7 would be categorized as "high confident" if the permuted singscore is <1.7.
A resistant profile with score -1.7 would be categorized as "completely incorrect" if the permuted singscore is >-1.7.

Completely wrong predictions are also inaccurate, and high confidence predictions are also accurate.

In [1]:
import pathlib
import pandas as pd

In [2]:
# Define paths
singscore_dir = pathlib.Path("..", "3.resistance-signature", "results", "singscore")

singscore_files = {
    "singscore_resultsbortezomib.tsv.gz": "initial",
    "singscore_results_valotherclones.tsv.gz": "validation",
    "singscore_results_otherclones.tsv.gz": "other",
    "singscore_results_LAST_BATCH_VALIDATIONotherclones.tsv.gz": "last_batch"
}

# Output files
output_dir = "results"
output_singscore_file = pathlib.Path(output_dir, "singscore_accuracy_per_sample.tsv")
output_singscore_summary_file = pathlib.Path(output_dir, "singscore_accuracy_summary.tsv")

In [3]:
# Load and process singscore results
full_singscore_df = []
for singscore_file, singscore_category in singscore_files.items():
    singscore_file = pathlib.Path(f"{singscore_dir}/{singscore_file}")
    singscore_df = pd.read_csv(singscore_file, sep="\t")
    full_singscore_df.append(singscore_df)

full_singscore_df = (
    pd.concat(full_singscore_df)
    .reset_index(drop=True)
    .drop(columns="Metadata_unique_sample_name") 
    .drop_duplicates(subset=["TotalScore", "Metadata_Plate", "Metadata_Well", "Metadata_batch"])
    .reset_index()
    .rename(
        columns={"index": "profile_id"}
    )
)

# Remove ixazomib and cb5083
full_singscore_df = full_singscore_df.query(
    "Metadata_dataset not in ['cb5083', 'ixazomib']"
)

# Drop samples from the training set
full_singscore_df = full_singscore_df.query(
    "Metadata_model_split != 'training'"
)

# Create a unique id per profile
full_singscore_df.profile_id = [f"profile_id_{x}" for x in full_singscore_df.profile_id]

full_singscore_df = full_singscore_df.loc[
    :, [
        "profile_id",
        "TotalScore",
        "min_permuted_value",
        "max_permuted_value",
        "Metadata_Plate",
        "Metadata_Well",
        "Metadata_batch",
        "Metadata_model_split",
        "Metadata_clone_number",
        "Metadata_dataset",
        "Metadata_clone_type",
    ]
]

all_pass_sign = []
all_pass_permuted = []
all_completely_incorrect = []  # Must pass oppositive permuted value
for profile_idx, profile in full_singscore_df.iterrows():
    if profile["Metadata_clone_type"] == "sensitive":
        if profile["TotalScore"] < profile["min_permuted_value"]:
            pass_sign = 1
            pass_permuted = 1
        else:
            pass_permuted = 0
            if profile["TotalScore"] < 0:
                pass_sign = 1
            else:
                pass_sign = 0
            
        completely_incorrect = 0
        if profile["TotalScore"] >= profile["max_permuted_value"]:
            completely_incorrect = 1

    if profile["Metadata_clone_type"] == "resistant":
        if profile["TotalScore"] > profile["max_permuted_value"]:
            pass_sign = 1
            pass_permuted = 1
        else:
            pass_permuted = 0
            if profile["TotalScore"] > 0:
                pass_sign = 1
            else:
                pass_sign = 0

        completely_incorrect = 0
        if profile["TotalScore"] <= profile["min_permuted_value"]:
            completely_incorrect = 1

    all_pass_sign.append(pass_sign)
    all_pass_permuted.append(pass_permuted)
    all_completely_incorrect.append(completely_incorrect)
    
full_singscore_df = full_singscore_df.assign(
    pass_sign = all_pass_sign,
    pass_permuted = all_pass_permuted,
    completely_incorrect = all_completely_incorrect
)

# Create a column indicating incorrect predictions
full_singscore_df = full_singscore_df.assign(incorrect = full_singscore_df.pass_sign + full_singscore_df.pass_permuted)
full_singscore_df.incorrect = ~(full_singscore_df.incorrect > 0)
full_singscore_df.incorrect = full_singscore_df.incorrect.astype(int)

# Recode many of the samples to the correct/consistent number
clone_recode_dict = {
    "WT clone 01": "WT001",
    "WT clone 02": "WT002",
    "WT clone 03": "WT003",
    "WT clone 04": "WT004",
    "WT clone 05": "WT005",
    "WT clone 10": "WT010",
    "WT clone 12": "WT012",
    "WT clone 13": "WT013",
    "WT clone 14": "WT014",
    "WT clone 15": "WT015",
}

full_singscore_df.Metadata_clone_number = full_singscore_df.Metadata_clone_number.replace(clone_recode_dict)

# Output to file
full_singscore_df.to_csv(output_singscore_file, index=False, sep="\t")

print(full_singscore_df.shape)
full_singscore_df.head()

(519, 15)


,profile_id,TotalScore,min_permuted_value,max_permuted_value,Metadata_Plate,Metadata_Well,Metadata_batch,Metadata_model_split,Metadata_clone_number,Metadata_dataset,Metadata_clone_type,pass_sign,pass_permuted,completely_incorrect,incorrect
0,profile_id_0,-0.151948,-0.154802,0.158931,219907,B02,2021_03_03_Batch12,test,WT_parental,bortezomib,sensitive,1,0,0,0
1,profile_id_1,-0.031256,-0.154802,0.158931,219907,B03,2021_03_03_Batch12,test,CloneA,bortezomib,resistant,0,0,0,1
2,profile_id_2,0.009995,-0.154802,0.158931,219907,B04,2021_03_03_Batch12,test,CloneE,bortezomib,resistant,1,0,0,0
5,profile_id_5,-0.185516,-0.154802,0.158931,219907,B07,2021_03_03_Batch12,test,WT_parental,bortezomib,sensitive,1,1,0,0
6,profile_id_6,-0.084243,-0.154802,0.158931,219907,B08,2021_03_03_Batch12,test,CloneA,bortezomib,resistant,0,0,0,1


In [4]:
# Summary of completely incorrect samples
all_sample_groups_df = (
    full_singscore_df
    .groupby(["Metadata_model_split", "Metadata_clone_number"])
    ["completely_incorrect"]
    .count()
    .reset_index()
    .rename(
        columns={"completely_incorrect": "total_samples"}
    )
)

confidently_incorrect_counts_df = (
    full_singscore_df
    .query("completely_incorrect == 1")
    .groupby(["Metadata_model_split", "Metadata_clone_number"])
    ["completely_incorrect"]
    .sum()
    .reset_index()
)

all_sample_groups_df = (
    all_sample_groups_df.merge(
        confidently_incorrect_counts_df,
        on=["Metadata_model_split", "Metadata_clone_number"],
        how="left"
    )
    .fillna(0)
)

all_sample_groups_df = (
    all_sample_groups_df.assign(
        proportion_completely_incorrect = all_sample_groups_df.completely_incorrect / all_sample_groups_df.total_samples
    )
    .sort_values("proportion_completely_incorrect", ascending=False)
    .reset_index(drop=True)
)

all_sample_groups_df.head(10)

,Metadata_model_split,Metadata_clone_number,total_samples,completely_incorrect,proportion_completely_incorrect
0,otherclone,BZ006,8,8.0,1.000000
1,otherclone,WT015,16,15.0,0.937500
2,otherclone,WT010,16,12.0,0.750000
3,inference,BZ007,3,2.0,0.666667
4,holdout,WT001,4,2.0,0.500000
5,inference,BZ006,3,1.0,0.333333
6,test,CloneA,24,6.0,0.250000
7,inference,WT_parental,6,1.0,0.166667
8,test,CloneE,24,3.0,0.125000
9,inference,WT012,3,0.0,0.000000


In [5]:
full_sample_summary_df = (
    all_sample_groups_df
    .groupby("Metadata_clone_number")
    [["total_samples", "completely_incorrect"]]
    .sum()
    .reset_index()
)

full_sample_summary_df.head(5)

,Metadata_clone_number,total_samples,completely_incorrect
0,BZ001,11,0.0
1,BZ002,10,0.0
2,BZ003,11,0.0
3,BZ004,11,0.0
4,BZ005,10,0.0


In [6]:
high_confidence_summary_df = (
    full_singscore_df
    .groupby(["Metadata_clone_number"])
    ["pass_permuted"]
    .sum()
    .reset_index()
    .rename(
        columns={"pass_permuted": "high_confidence"}
    )
)

high_confidence_summary_df.head()

,Metadata_clone_number,high_confidence
0,BZ001,3
1,BZ002,4
2,BZ003,8
3,BZ004,1
4,BZ005,3


In [7]:
correct_summary_df = (
    full_singscore_df
    .groupby(["Metadata_clone_number"])
    ["pass_sign"]
    .sum()
    .reset_index()
    .rename(
        columns={"pass_sign": "accurate"}
    )
)

correct_summary_df.head()

,Metadata_clone_number,accurate
0,BZ001,9
1,BZ002,9
2,BZ003,11
3,BZ004,9
4,BZ005,9


In [8]:
incorrect_summary_df = (
    full_singscore_df
    .groupby(["Metadata_clone_number"])
    ["incorrect"]
    .sum()
    .reset_index()
)

incorrect_summary_df.head()

,Metadata_clone_number,incorrect
0,BZ001,2
1,BZ002,1
2,BZ003,0
3,BZ004,2
4,BZ005,1


In [9]:
overall_summary_df = full_sample_summary_df.merge(
    high_confidence_summary_df,
    on="Metadata_clone_number"
).merge(
    correct_summary_df,
    on="Metadata_clone_number"
).merge(
    incorrect_summary_df,
    on="Metadata_clone_number"
)

overall_summary_df.head()

,Metadata_clone_number,total_samples,completely_incorrect,high_confidence,accurate,incorrect
0,BZ001,11,0.0,3,9,2
1,BZ002,10,0.0,4,9,1
2,BZ003,11,0.0,8,11,0
3,BZ004,11,0.0,1,9,2
4,BZ005,10,0.0,3,9,1


In [10]:
# Calculate proportions
overall_summary_df = (
    overall_summary_df.assign(
        prop_completely_incorrect = overall_summary_df.completely_incorrect / overall_summary_df.total_samples,
        prop_high_confidence = overall_summary_df.high_confidence / overall_summary_df.total_samples,
        prop_accurate = overall_summary_df.accurate / overall_summary_df.total_samples,
        prop_inaccurate = overall_summary_df.incorrect / overall_summary_df.total_samples
    )
    .sort_values("prop_completely_incorrect", ascending=False)
    .reset_index(drop=True)
)

overall_summary_df.to_csv(output_singscore_summary_file, index=False, sep="\t")

overall_summary_df.head()

,Metadata_clone_number,total_samples,completely_incorrect,high_confidence,accurate,incorrect,prop_completely_incorrect,prop_high_confidence,prop_accurate,prop_inaccurate
0,BZ006,11,9.0,0,0,11,0.818182,0.000000,0.000000,1.000000
1,WT015,19,15.0,0,1,18,0.789474,0.000000,0.052632,0.947368
2,WT010,19,12.0,0,1,18,0.631579,0.000000,0.052632,0.947368
3,WT001,10,2.0,3,6,4,0.200000,0.300000,0.600000,0.400000
4,BZ007,11,2.0,8,8,3,0.181818,0.727273,0.727273,0.272727
